In [1]:
    import abc, jsonschema, rdflib, collections, copy, pyld.jsonld as jsonld, ast, pathlib
    import strict_rfc3339, rfc3986, rfc3987, IPython

In [2]:
    class Schema:
        @classmethod
        def schema(cls): return {**collections.ChainMap(*(getattr(object, '__annotations__', {}) or {} for object in cls.__mro__)), 'title': cls.__name__, }
        @classmethod
        def validate(cls, object, schema=None): jsonschema.validate(object, schema or cls.schema(), format_checker=jsonschema.FormatChecker())

In [3]:
    class JsonSchemaMeta(Schema, abc.ABCMeta):
        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = cls.validate(kwargs) or kwargs

    class JsonSchema(JsonSchemaMeta, **jsonschema.Draft7Validator.META_SCHEMA): 
        def __new__(cls, object=None, *args, **kwargs):
            object= super().__new__(cls, object, *args, **kwargs)
            return object

In [4]:
    class Pandas:
        def frame(self): return __import__('pandas').DataFrame(self)
        def series(self): return __import__('pandas').Series(self)        

In [5]:
    class Type(Schema, metaclass=JsonSchema): 
        __annotations__ = {}
        def __new__(cls, object=None, *args, **kwargs):
            if object is None: object = copy.copy(cls.schema().get('default'))
            object = super().__new__(cls, object, *args, **kwargs)
            return cls.validate(object) or object

        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = type(cls).validate(kwargs) or kwargs

        @classmethod
        def object(type, object=None):
            for cls in (type.__subclasses__()):
                try: return cls.object(object)
                except BaseException as e: ...
            else: return type(object)
            
        def pipe(self, callable, *args, **kwargs): return callable(self, *args, **kwargs)
    class Dict(Pandas, Type, dict, type='object', default={}): 
        def valmap(self, callable, *args, **kwargs): return Dict({k: callable(x, *args, **kwargs) for k, x in self.items()})
        def keymap(self, callable, *args, **kwargs): return Dict({callable(x, *args, **kwargs): v for x, v in self.items()})
        def itemmap(self, callable, *args, **kwargs): return Dict(callable(x, *args, **kwargs) for x in self.items())
        def valfilter(self, callable, *args, **kwargs): return Dict({k: v for k, v in self.items() if callable(v, *args, **kwargs)})
        def keyfilter(self, callable, *args, **kwargs): return Dict({k: v for k, v in self.items() if callable(k, *args, **kwargs)})
        def itemfilter(self, callable, *args, **kwargs): return Dict(x for x in self.items() if callable(x, *args, **kwargs))
        def __getitem__(self, object): return Type.object(super().__getitem__(object))
    
    class List(Pandas, Type, list, type='array', default=[]): 
        def map(self, callable, *args, **kwargs): return List(map(lambda x: callable(x, *args, **kwargs), self))
        def filter(self, callable, *args, **kwargs): return List(filter(lambda x: callable(x, *args, **kwargs), self))
        def __getitem__(self, object): return Type.object(super().__getitem__(object))

    class Null(Type, type='null'): 
        def __new__(cls, object=None, *args, **kwargs): return cls.validate(cls, object)
    class Integer(Type, int, type='integer', default=''): ...
    class Number(Type, float, type='number', default=''): ...
    
    class String(Type, str, type='string', default=''): 
        def imp(self, *args, **kwargs): return __import__('importlib').import_module(self, *args, **kwargs)
        def text(self): return str(self)
        def yaml(self): 
            try: from ruamel import yaml
            except: import yaml
            return Type.object(yaml.safe_load(self.text()))
        def json(self): 
            try: import ujson as json
            except: import json
            return json.loads(self.text())
        def path(self): return pathlib.Path(self)
        
    class _time_attrs:
        def pandas(self): return __import__('pandas').to_datetime(self)
        def arrow(self): return __import__('arrow').get(self)
    
    class Date(String, _time_attrs, format='date'): ...
    class Datetime(String, _time_attrs, format='date-time'): ...
    class Time(String, _time_attrs, format='time'): ...
    class Color(String, format='color'): ...
    
    class Email(String, format='email'): ...
    class Uri(String, format='uri'):         
        def text(self, **object): return __import__('requests').get(self, **object).text
    
        
    class File(String): 
        def text(self): return self.path().read_text()
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().exists(): return 
            raise ValueError(F"{object} is not a file.")
    class Dir(File): 
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().is_dir(): return 
            raise ValueError(F"{object} is not a file.")

In [6]:
    class StringTypes(ast.NodeTransformer):
        def visit_Str(self, node):
            next = ast.parse(F"""__import__('importlib').import_module('{__name__}').String.object('')""").body[0].value
            next.args = [node]
            return ast.copy_location(next, node)
        visit_JoinedStr = visit_Str

In [7]:
    def unload_ipython_extension(shell): shell.ast_transformers = [x for x in shell.ast_transformers if not isinstance(x, StringTypes)]
    def load_ipython_extension(shell): unload_ipython_extension(shell) or shell.ast_transformers.append(StringTypes())
    __name__ == '__main__' and load_ipython_extension(get_ipython())

In [11]:
    __name__ == '__main__' and "jschema.ipynb".yaml()['cells'].series().apply("pandas".imp().Series).T

,0,1,2,3,4,5,6,7,8
cell_type,code,code,code,code,code,code,code,code,code
execution_count,1,2,3,4,5,6,7,10,NaN
metadata,{},{},{},{},{},{},{},{},{}
outputs,[],[],[],[],[],[],[],"[{'data': {'text/html': ['<div>\n', '<style sc...",[]
source,"[ import abc, jsonschema, rdflib, collectio...","[ class Schema:\n, @classmethod\n, ...","[ class JsonSchemaMeta(Schema, abc.ABCMeta)...","[ class Pandas:\n, def frame(self):...","[ class Type(Schema, metaclass=JsonSchema):...",[ class StringTypes(ast.NodeTransformer):\n...,[ def unload_ipython_extension(shell): shel...,"[ __name__ == '__main__' and ""jschema.ipynb...",[]
